In [5]:
import matplotlib.pyplot as plt
import nibabel
import numpy as np
import pandas as pd
from pathlib import Path
from skimage.transform import rescale
from tqdm import tqdm
import os
import researchpy

In [6]:
import scipy
def compute_kl(img, brain, mask, _bin_heuristics = 'sturges'):
    size, _bins = np.histogram(img[brain.astype(bool)], bins = _bin_heuristics)
    bins = [(_bins[i]+_bins[i+1])/2 for i in range(len(_bins)-1)]
    # WT
    #size_healthy, bin_edges = np.histogram(img[brain.astype(bool)^mask.astype(bool)].reshape(-1), bins=_bins)
    # TC
    size_healthy, bin_edges = np.histogram(img[brain.astype(bool)^(mask == 2)].reshape(-1), bins=_bins)
    # ET
    size_tumor, _ = np.histogram(img[mask.astype(bool)].reshape(-1), bins = _bins)
    size_healthy = np.round(size_healthy/size_healthy.sum(), 5)
    size_tumor = np.round(size_tumor/size_tumor.sum(), 5)
    
    size_healthy = np.where(size_healthy>10e-6, size_healthy, 10e-6)
    size_tumor = np.where(size_tumor>10e-6, size_tumor, 10e-6)

    kl_dist = 0
    for h,t in zip(size_healthy, size_tumor):
        kl_dist += h * np.log2(h/t)

#     kl_dist = scipy.spatial.distance.jensenshannon(size_healthy, size_tumor)            
    return kl_dist, size_healthy, size_tumor, _bins

In [53]:
import numpy as np
from scipy import stats
import math

def coef_of_var(img, brain, mask):
    """
    where σ is standard deviation, and µ is mean of a region
    of interest, which was the prostate for the purposes of this study.
    """
    tumor_roi = img[brain.astype(bool)^mask.astype(bool)].reshape(-1)
    return np.round((np.std(tumor_roi)/np.mean(tumor_roi)),3)

def snr_roi(img, brain, mask):
    """
    where σ is standard deviation, and µ is mean of a region
    of interest, which was the prostate for the purposes of this study.
    """
    tumor_roi = img[brain.astype(bool)^mask.astype(bool)].reshape(-1)
    return np.round(np.mean(tumor_roi)/np.std(tumor_roi),3)
    #return np.round(np.std(tumor_roi),3)


def calculate_psnr(img1, img2, brain, mask):
    """ img1 - w/o noise, img2 with noise
    implementation from here https://cvnote.ddlee.cc/2019/09/12/psnr-ssim-python"""
    tumor_roi1 = img1[brain.astype(bool)^mask.astype(bool)].reshape(-1)
    tumor_roi2 = img2[brain.astype(bool)^mask.astype(bool)].reshape(-1)
    mse = np.mean((tumor_roi1 - tumor_roi2)**2)
    if mse == 0:
        return float('inf')
    return 20 * math.log10(255.0 / math.sqrt(mse))


def diff_of_modes(img, img_hist, brain):
    
    """
    where ω test is the principal mode of the intensity distribution
    of an MR image undergoing IS, and ω
    temp is the principal mode of the template intensity distribution that is being
    standardized against. 
    """
    brain_test = img[brain.astype(bool)].reshape(-1)
    brain_temp = img_hist[brain.astype(bool)].reshape(-1)
    
    diff = abs(stats.mode(brain_test) - stats.mode(brain_temp))/stats.mode(brain_temp)
    return np.round(diff, 3)

from researchpy import ttest

def ttest_pair(df_1, df_2, name_1 = 'one', name_2 = 'two', correction = None):
    return ttest(pd.Series(df_1), pd.Series(df_2), 
#                  group1_name = name_1,
#                  group2_name= name_2, 
                 equal_variances=False, paired=True,)[1].iloc[4].values[1]

### 4a resamp to 4b N4: Coef of variance

a. T2
b. CT1

In [82]:
dataset = 'gbm'
main_img = 'CT1.nii.gz'
label_name = 'CT1_SEG.nii.gz'
mask_name = 'CT1_mask.nii.gz'

root = Path('/anvar/public_datasets/preproc_study/{}/4a_resamp/'.format(dataset)) 
root_test = Path('/anvar/public_datasets/preproc_study/{}/6_hist/6_hist_fold_0/'.format(dataset))
root_brain = Path('/anvar/public_datasets/preproc_study/{}/5_ss_shared/'.format(dataset))

all_orig = []
all_bfc = []

for patient in tqdm(root.glob('*')):
    if patient.is_dir():
        img = nibabel.load(patient / main_img).get_fdata()
        mask = nibabel.load(patient / label_name).get_fdata()
        brain = nibabel.load(str(root_brain) + '/' +
                             str(patient).split('/')[-1] +'/' + mask_name).get_fdata()
        
        img_temp = nibabel.load(str(root_test) +'/' +
                             str(patient).split('/')[-1] +'/' + main_img).get_fdata()
        
        coef_orig = coef_of_var(img, brain, mask)
        all_orig.append(coef_orig)
        
        coef_bfc = coef_of_var(img_temp, brain, mask)
        all_bfc.append(coef_bfc)

102it [01:48,  1.06s/it]


In [83]:
from researchpy import ttest

def ttest_pair(df_1, df_2, name_1 = 'one', name_2 = 'two', correction = None):
    return ttest(pd.Series(df_1), pd.Series(df_2), 
#                  group1_name = name_1,
#                  group2_name= name_2, 
                 equal_variances=False, paired=True,)[1].iloc[4].values[1]

In [84]:
# GBM CT1 46_hist
np.array(all_orig).mean(), np.array(all_orig).std(), np.array(all_bfc).mean(), np.array(all_bfc).std(), ttest_pair(all_orig, all_bfc)

/home/kate/miniconda3/lib/python3.9/site-packages/researchpy/ttest.py:38: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  groups = group1.append(group2, ignore_index= True)


(0.24193137254901956,
 0.062488000385944154,
 0.3103725490196078,
 0.11983173837073999,
 0.0008)

In [81]:
# GBM CT1 4d_susan
np.array(all_orig).mean(), np.array(all_orig).std(), np.array(all_bfc).mean(), np.array(all_bfc).std(), ttest_pair(all_orig, all_bfc)

/home/kate/miniconda3/lib/python3.9/site-packages/researchpy/ttest.py:38: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  groups = group1.append(group2, ignore_index= True)


(0.24193137254901956,
 0.062488000385944154,
 0.23681372549019614,
 0.061749927425877316,
 0.0)

In [22]:
# GBM T2 4b_n4
np.array(all_orig).mean(), np.array(all_orig).std(), np.array(all_bfc).mean(), np.array(all_bfc).std()

(0.3333137254901961,
 0.0753851522792117,
 0.31238235294117656,
 0.07588355942193341)

In [24]:
# GBM CT1 4b_n4
np.array(all_orig).mean(), np.array(all_orig).std(), np.array(all_bfc).mean(), np.array(all_bfc).std()

(0.24193137254901956,
 0.062488000385944154,
 0.22451960784313726,
 0.06060119237682763)

### 4a resamp to 4d SUSAN: SNR

In [94]:
import scipy
def snr_roi(img, brain, mask):
    """
    where σ is standard deviation, and µ is mean of a region
    of interest, which was the prostate for the purposes of this study.
    
    if calculated for tumor region - it deacrease everything
    """
    tumor_roi = img[brain.astype(bool)].reshape(-1)
    return np.round(np.mean(tumor_roi)/np.std(tumor_roi),3)
    #return np.round(np.std(tumor_roi),3)

In [99]:
dataset = 'gbm'
main_img = 'CT1.nii.gz'
label_name = 'CT1_SEG.nii.gz'
mask_name = 'CT1_mask.nii.gz'

root = Path('/anvar/public_datasets/preproc_study/{}/4a_resamp/'.format(dataset)) 
root_test = Path('/anvar/public_datasets/preproc_study/{}/6_hist/6_hist_fold_0/'.format(dataset))
root_brain = Path('/anvar/public_datasets/preproc_study/{}/5_ss_shared/'.format(dataset))

all_orig = []
all_nf = []

for patient in tqdm(root.glob('*')):
    if patient.is_dir():
        img = nibabel.load(patient / main_img).get_fdata()
        mask = nibabel.load(patient / label_name).get_fdata()
        brain = nibabel.load(str(root_brain) + '/' +
                             str(patient).split('/')[-1] +'/' + mask_name).get_fdata()
        
        img_temp = nibabel.load(str(root_test) +'/' +
                             str(patient).split('/')[-1] +'/' + main_img).get_fdata()
        
        coef_orig = snr_roi(img, brain, mask)
        all_orig.append(coef_orig)
        
        coef_nf = snr_roi(img_temp, brain, mask)
        all_nf.append(coef_nf)

102it [01:26,  1.18it/s]


In [100]:
# GBM CT1 hist
np.array(all_orig).mean(), np.array(all_orig).std(), np.array(all_nf).mean(), np.array(all_nf).std(), ttest_pair(all_orig, all_nf)

/home/kate/miniconda3/lib/python3.9/site-packages/researchpy/ttest.py:38: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  groups = group1.append(group2, ignore_index= True)


(4.20636274509804,
 0.9766134843569172,
 3.524647058823529,
 1.1610088382485937,
 0.001)

In [98]:
# GBM CT1 4b_n4
np.array(all_orig).mean(), np.array(all_orig).std(), np.array(all_nf).mean(), np.array(all_nf).std(), ttest_pair(all_orig, all_nf)

/home/kate/miniconda3/lib/python3.9/site-packages/researchpy/ttest.py:38: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  groups = group1.append(group2, ignore_index= True)


(4.20636274509804,
 0.9766134843569172,
 4.568303921568627,
 1.0966708873142665,
 0.0)

In [96]:
# GBM CT1 4d_susan
np.array(all_orig).mean(), np.array(all_orig).std(), np.array(all_nf).mean(), np.array(all_nf).std(), ttest_pair(all_orig, all_nf)

/home/kate/miniconda3/lib/python3.9/site-packages/researchpy/ttest.py:38: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  groups = group1.append(group2, ignore_index= True)


(4.20636274509804,
 0.9766134843569172,
 4.2941078431372555,
 1.003528431036844,
 0.0)

In [86]:
# GBM CT1
np.array(all_orig).mean(), np.array(all_orig).std(), np.array(all_nf).mean(), np.array(all_nf).std(), ttest_pair(all_orig, all_nf)

/home/kate/miniconda3/lib/python3.9/site-packages/researchpy/ttest.py:38: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  groups = group1.append(group2, ignore_index= True)


(129.08098039215685,
 153.4854274961186,
 127.78495098039217,
 153.68665577040107,
 0.0)

In [33]:
# GBM T2
np.array(all_orig).mean(), np.array(all_orig).std(), np.array(all_nf).mean(), np.array(all_nf).std()

(169.56006862745102, 105.93496562479578, 168.4317058823529, 106.1864894376397)

## PSNR

In [112]:
dataset = 'gbm'
main_img = 'CT1.nii.gz'
label_name = 'CT1_SEG.nii.gz'
mask_name = 'CT1_mask.nii.gz'

root = Path('/anvar/public_datasets/preproc_study/{}/4a_resamp/'.format(dataset)) 
root_test = Path('/anvar/public_datasets/preproc_study/{}/4b_n4/'.format(dataset))
root_brain = Path('/anvar/public_datasets/preproc_study/{}/5_ss_shared/'.format(dataset))

all_psnr = []

for patient in tqdm(root.glob('*')):
    if patient.is_dir():
        img = nibabel.load(patient / main_img).get_fdata()
        mask = nibabel.load(patient / label_name).get_fdata()
        brain = nibabel.load(str(root_brain) + '/' +
                             str(patient).split('/')[-1] +'/' + mask_name).get_fdata()
        
        img_temp = nibabel.load(str(root_test) +'/' +
                             str(patient).split('/')[-1] +'/' + main_img).get_fdata()
        
        coef_psnr = calculate_psnr(img, img_temp, brain, mask)
        all_psnr.append(coef_psnr)

102it [01:20,  1.26it/s]


In [ ]:
#susan
np.array(all_psnr).mean(), np.array(all_psnr).std()

In [ ]:
#n4
np.array(all_psnr).mean(), np.array(all_psnr).std()

In [111]:
#hist
np.array(all_psnr).mean(), np.array(all_psnr).std()

(-1.5115249944443252, 9.350359089366194)

In [ ]:
## could be checked from here https://scikit-image.org/docs/stable/api/skimage.metrics.html

### 4a resamp to 6 hist: diff of modes and KL to the mean hist

In [54]:
from torchio.transforms import HistogramStandardization

base_dir = '/anvar/public_datasets/preproc_study/gbm/4a_resamp/'
save_dir = '/anvar/public_datasets/preproc_study/gbm/6_hist/6_hist_fold_0/'
mask_name = 'CT1_SEG.nii.gz'

# Separate dataset for each fold
# Create dataset
temp_t1_list = []
temp_t2_list = []
temp_ct1_list = []
temp_fl_list = []

for patient in os.listdir(base_dir):
    if os.path.isdir(base_dir + patient):
        temp_t1_list.append(base_dir + patient + '/T1.nii.gz')
        temp_t2_list.append(base_dir + patient + '/T2.nii.gz')
        temp_ct1_list.append(base_dir + patient + '/CT1.nii.gz')
        temp_fl_list.append(base_dir + patient + '/FLAIR.nii.gz')
#             print(len(subjects_list))

print('For landmarks there are ', len(temp_t1_list))
# logging.info("Training T1 landmarks started.")
t1_landmarks = HistogramStandardization.train(temp_t1_list)
# logging.info("Training T2 landmarks started.")
t2_landmarks = HistogramStandardization.train(temp_t2_list)
# logging.info("Training CT1 landmarks started.")
ct1_landmarks = HistogramStandardization.train(temp_ct1_list)
# logging.info("Training FLAIR landmarks started.")
fl_landmarks = HistogramStandardization.train(temp_fl_list)

# Saving landmarks
landmarks_dict = {
't1': t1_landmarks,
't2': t2_landmarks,
'ct1': ct1_landmarks,
'fl': fl_landmarks
}

For landmarks there are  102


In [59]:
import torchio as tio
hist_standardization = tio.HistogramStandardization(landmarks_dict)

In [69]:
### standardizing one template subject
patient = 'TCGA-02-0086'
subject = tio.Subject(
            t1 = tio.ScalarImage(base_dir + patient + '/T1.nii.gz'),
            t2 = tio.ScalarImage(base_dir + patient + '/T2.nii.gz'),
            # can be commented for other datasets
            ct1 = tio.ScalarImage(base_dir + patient + '/CT1.nii.gz'),
            fl = tio.ScalarImage(base_dir + patient + '/FLAIR.nii.gz')
        )
# hist standartize for the four landmarks
hist_standardize = hist_standardization(subject)
# saving forewer here in the folder
hist_standardize['t1'].save('T1_gbm_hist.nii.gz')
hist_standardize['t2'].save('T2_gbm_hist.nii.gz')
hist_standardize['fl'].save('FLAIR_gbm_hist.nii.gz')
hist_standardize['ct1'].save('CT1_gbm_hist.nii.gz')

hist_temp = nibabel.load('CT1_gbm_hist.nii.gz').get_fdata()
brain_hist = nibabel.load(str(root_brain) + '/' + str(patient).split('/')[-1] +'/' + mask_name).get_fdata()
img_hist_mode = stats.mode(hist_temp[brain_hist.astype(bool)].reshape(-1), keepdims = False)

### diff of modes

In [72]:
def diff_of_modes(img, img_hist_mode, brain):
    
    """
    where ω test is the principal mode of the intensity distribution
    of an MR image undergoing IS, and ω
    temp is the principal mode of the template intensity distribution that is being
    standardized against. 
    """
    brain_test = img[brain.astype(bool)].reshape(-1)
    
    diff = abs(stats.mode(brain_test, keepdims = False)[0] - img_hist_mode[0])/img_hist_mode[0]
    return np.round(diff, 3)

In [103]:
dataset = 'gbm'
main_img = 'CT1.nii.gz'
label_name = 'CT1_SEG.nii.gz'
mask_name = 'CT1_mask.nii.gz'

root = Path('/anvar/public_datasets/preproc_study/{}/4a_resamp/'.format(dataset)) 
root_test = Path('/anvar/public_datasets/preproc_study/{}/4b_n4'.format(dataset))
root_brain = Path('/anvar/public_datasets/preproc_study/{}/5_ss_shared/'.format(dataset))

all_orig = []
all_hist = []

for patient in tqdm(root.glob('*')):
    if patient.is_dir():
        img = nibabel.load(patient / main_img).get_fdata()
        mask = nibabel.load(patient / label_name).get_fdata()
        brain = nibabel.load(str(root_brain) + '/' +
                             str(patient).split('/')[-1] +'/' + mask_name).get_fdata()
        
        img_temp = nibabel.load(str(root_test) +'/' +
                             str(patient).split('/')[-1] +'/' + main_img).get_fdata()
        
        coef_orig = diff_of_modes(img, img_hist_mode, brain)
        all_orig.append(coef_orig)
        
        coef_hist = diff_of_modes(img_temp, img_hist_mode, brain)
        all_hist.append(coef_hist)

0it [00:00, ?it/s]/tmp/ipykernel_12306/3912426824.py:11: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  diff = abs(stats.mode(brain_test)[0] - img_hist_mode[0])/img_hist_mode[0]
102it [01:53,  1.12s/it]


In [104]:
# GBM CT1 4b_n4
np.array(all_orig).mean(), np.array(all_orig).std(), np.array(all_hist).mean(), np.array(all_hist).std(), ttest_pair(all_orig, all_hist)

/home/kate/miniconda3/lib/python3.9/site-packages/researchpy/ttest.py:38: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  groups = group1.append(group2, ignore_index= True)


(19.696058823529416,
 21.770764256908237,
 20.43320588235294,
 22.11269927667696,
 0.0076)

In [102]:
# GBM CT1 4d_susan
np.array(all_orig).mean(), np.array(all_orig).std(), np.array(all_hist).mean(), np.array(all_hist).std(), ttest_pair(all_orig, all_hist)

/home/kate/miniconda3/lib/python3.9/site-packages/researchpy/ttest.py:38: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  groups = group1.append(group2, ignore_index= True)


(19.696058823529416,
 21.770764256908237,
 21.487970588235296,
 24.84921863783446,
 0.0111)

In [76]:
# GBM CT1
np.array(all_orig).mean(), np.array(all_orig).std(), np.array(all_hist).mean(), np.array(all_hist).std()

(19.696058823529416,
 21.770764256908237,
 0.6849019607843135,
 0.3470235769185313)

### KL of mean hist

4-4b SNR, 4-4a средняя гистограмма по 4а ресемп (для неё нужно зафиксировать бины, 102 + 1 гистограмма для датасета). Гистограамы должны быть нормализованы (суммироваться в единицу normalize true работает правильно) 102 и разница от среднего. И среднее расстояние после Hist Matching.

In [ ]:
dataset = 'gbm'
main_img = '/FLAIR.nii.gz'
label_name = 'mask_GTV_FLAIR.nii.gz'
mask_name = 'FLAIR_mask.nii.gz'
root = '/anvar/public_datasets/preproc_study/{}/6_hist/6_hist_fold_0/'.format(dataset)
root_orig = '/anvar/public_datasets/preproc_study/{}/4a_resamp/'.format(dataset)

def calc_cdf(t1_image, bins = 100):
    """ Takes np.array 3D size as input
    Outputting CDFs and cumsum of the normalised and original images"""

    masked_img = t1_image.reshape(-1)
    t1_hist, t1_hist_bins = np.histogram(masked_img, bins= bins)
    cumsum = np.cumsum(t1_hist)
    return t1_hist_bins[:-1], (cumsum- cumsum.min())/cumsum.sum()

plt.figure(figsize = [10,4])

for patient in tqdm.tqdm(os.listdir(root)[30:70]):
        img = sitk.GetArrayFromImage(sitk.ReadImage(root + patient + main_img))
        brain_mask = sitk.GetArrayFromImage(sitk.ReadImage('/anvar/public_datasets/preproc_study/{}/5_ss_shared/'.format(dataset) +
                             str(patient).split('/')[-1] +'/' + mask_name))
        bins_cdf, cdf =  calc_cdf(img[brain_mask.astype(bool)], bins = 100)
        plt.plot(bins_cdf, cdf, '-', drawstyle='steps')
plt.show()